In [1]:
import torch
from torchvision.models import squeezenet1_1, resnet18, vgg16_bn
import random
import numpy as np
from modules.utils.data_gen import make_regular_loaders
import time

In [2]:
def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)  

In [3]:
def train_model(model, train_loader, val_loader, device, model_name):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)  
    num_epochs = 250

    for epoch in range(num_epochs):
        start_time = time.time() 
        model.train()
        train_loss = 0.0
        print(f"Starting epoch {epoch}")
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)

        avg_train_loss = train_loss / len(train_loader.dataset)
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():

            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        avg_val_loss = val_loss / len(val_loader.dataset)
        accuracy = 100 * correct / total

        scheduler.step()  

        print(
            f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Valid Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.2f}%, LR: {scheduler.get_last_lr()[0]}"
        )

        if (epoch + 1) % 10 == 0:
            torch.save(model.state_dict(), f"training_checkpoints/{model_name}/{model_name}_epoch_{epoch+1}.pth")
    
    end_time = time.time()  
    duration = end_time - start_time
    print(f"Training time for {model_name}: {duration:.2f} seconds")

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
num_classes = 70
model_squeeze = squeezenet1_1(pretrained=True)
model_squeeze.classifier[1] = torch.nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
model_squeeze.classifier[3] = torch.nn.AdaptiveAvgPool2d((1, 1))
model_squeeze.to(device)

model_resnet = resnet18(pretrained=True)
model_resnet.fc = torch.nn.Linear(model_resnet.fc.in_features, num_classes)
model_resnet.to(device)
train_loader, val_loader, _ = make_regular_loaders()

cuda


c:\Users\jack\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jack\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\jack\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_W

In [5]:
train_model(model_resnet, train_loader, val_loader, device, "resnet")
train_model(model_squeeze, train_loader, val_loader, device, "squeezenet")

Starting epoch 0


KeyboardInterrupt: 